In [8]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pickle


In [9]:
class Autoencoder(nn.Module):
    def __init__(self, input_size):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 4096),
            nn.BatchNorm1d(4096),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(4096, 2048),
            nn.ReLU(True),
            nn.Linear(2048, 1024),
            nn.ReLU(True),



        )
        self.decoder = nn.Sequential(
            nn.Linear(1024, 2048),
            nn.BatchNorm1d(2048),
            nn.ReLU(True),
            nn.Linear(2048, 4096),
            nn.BatchNorm1d(4096),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(4096, input_size),
            nn.Sigmoid()
        )

    # initialize weights using xavier normal
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, input_size):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 4096),
            nn.BatchNorm1d(4096),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(4096, 2048),
            nn.ReLU(True),
            nn.Linear(2048, 1024),
            nn.ReLU(True),



        )
        self.decoder = nn.Sequential(
            nn.Linear(1024, 2048),
            nn.BatchNorm1d(2048),
            nn.ReLU(True),
            nn.Linear(2048, 4096),
            nn.BatchNorm1d(4096),
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(4096, input_size),
            nn.Sigmoid()
        )

    # initialize weights using xavier normal
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [10]:
class VectorDataset(Dataset):
    def __init__(self, vector_dict):
        self.vectors = list(vector_dict.values())
        self.keys = list(vector_dict.keys())

    def __len__(self):
        return len(self.vectors)

    def __getitem__(self, idx):
        vector = self.vectors[idx]
        return torch.tensor(vector, dtype=torch.float)

In [11]:
# load feature vectors
# Load the feature vectors
with open('data/midi_feature_vectors.pkl', 'rb') as f:
    feature_vectors = pickle.load(f)


dataset = VectorDataset(feature_vectors)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)  # Adjust batch size as needed


In [24]:
feature_vectors['no woman no cry the fugees']

array([ 0.28346457,  0.27559055,  0.25984252, ..., 54.89547098,
        0.0683147 , 56.62619607])

In [12]:
input = len(feature_vectors['no woman no cry the fugees'])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
autoencoder = Autoencoder(input_size=input).to(device)
autoencoder.init_weights()

In [ ]:
criterion = nn.MSELoss()  # Using MSE loss for reconstruction error
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=1e-4)  # Adjust learning rate as needed
# Example of setting a learning rate scheduler
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10000, gamma=0.1)

num_epochs = 100 # Adjust the number of epochs as needed
autoencoder.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for data in dataloader:
        # Transfer data to GPU
        inputs = data.to(device)
        
        # Forward pass
        outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # save loss for average
        running_loss += loss.item()



    # average loss
    run = running_loss / len(dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {run:.4f}')




In [20]:
# generate the embeddings
autoencoder.eval()
embeddings = {}
for key, vector in feature_vectors.items():
    vector = torch.tensor([vector], dtype=torch.float).to(device)
    embedding = autoencoder.encoder(vector).detach().cpu().numpy()
    embeddings[key] = embedding

C:\Users\Administrator\AppData\Local\Temp\ipykernel_1516\3887502770.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\b\abs_f0dma8qm3d\croot\pytorch_1669187301762\work\torch\csrc\utils\tensor_new.cpp:204.)
  vector = torch.tensor([vector], dtype=torch.float).to(device)


In [22]:
# save embeddings
with open('data/midi_embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings, f)

In [15]:
# Save the model
torch.save(autoencoder.state_dict(), 'models/midi_autoencoder.pth')